# Weighted gene correlation network analysis (WGCNA)
**(applied to DNA methylation data)**

This notebook was prepared by [Evgeniy Efimov](https://www.linkedin.com/in/evgeniy-efimov-84152725b) for the course on Computational Biology of Aging.

In [ ]:
# Uncomment if you work using Google Colab

# The notebook will restart automatically after this cell is executed
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [ ]:
# import condacolab
# condacolab.check()

In [ ]:
# !wget -nv https://raw.githubusercontent.com/GentlemanOfFate/computational_aging_course/main/environment_meth.yml

# !mamba env update -n base -f environment_meth.yml

In [ ]:
# !wget -r -np -R -nv https://raw.githubusercontent.com/GentlemanOfFate/computational_aging_course/main/meth/data

## Initialization

In [ ]:
import numpy as np
import pandas as pd

import PyWGCNA
import biomart

import seaborn as sns

data_dir = './data'

In this practice, we will perform WGCNA for the dataset of *in vitro* cellular reprogramming in human fibroblasts. The article by Ohnuki *et al.* can be found at https://doi.org/10.1073/pnas.1413299111, and the whole dataset is deposited as [GSE54848](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE54848). We have downloaded and pre-processed the data of interest from this set, which includes samples of DNA methylation starting from normal fibroblasts, going through several days of reprogramming, and ending with induced pluripotent stem cells (iPSCs). Human embryonic stem cells (hESCs) are also included for comparison.

# Part I. Building gene correlation network

## Setting up the PyWGCNA object

In [ ]:
metLev = pd.read_pickle(f'{data_dir}/reprog_metlevs.pkl.gz')

In [ ]:
# PyWGCNA was developed for gene expression data, so don't be surprised by such names of variables as geneExp or geneExpr

pyWGCNA_reprog = PyWGCNA.WGCNA(name='reprog_metlevs', 
                               species='homo sapiens', 
                               geneExp=metLev,
                               outputPath='wgcna_reprog_',
                               TPMcutoff=0, # since we're working with DNAm, this cutoff is unrelatable and should be zeroed
                               save=True)
pyWGCNA_reprog.geneExpr.to_df()

If we were working with gene expression data, we would next use the `preprocess()` function to remove lowly-expressed genes (with expression levels less than `TPMcutoff`) and genes with too many missing values, and to perform hierarchical clustering (to see if there are any obvious outliers). In our case however, there is no such notion as lowly-methylated sites, and there are no missing values in the data.

Therefore, we actually use the `processing()` function only to do the clustering part:

In [ ]:
pyWGCNA_reprog.preprocess()

By default, no samples are removed after clustering (at initialization, `pyWGCNA_reprog.cut = inf`)

```{admonition} Question 1
:class: dropdown
Describe what you can infer from this dendrogram. Should we exclude any samples here? If yes, why? **(0.25 pts)**
```

## Constructing gene network modules

Now we get to construct the gene network and identify its modules

PyWGCNA function findModules performs all of the required steps in one go:

1. Choosing the soft-thresholding power: analysis of network topology
2. Co-expression similarity and adjacency
3. Topological Overlap Matrix (TOM)
4. Clustering using TOM
5. Merging of modules whose expression profiles are very similar

In [ ]:
pyWGCNA_reprog.findModules()

...

Oops, something's gone wrong. The trouble is, PyWGCNA employs a very memory-consuming approach to calculating network connectivity. It is still manageable with gene expression matrices, but with almost 500k sites it becomes unbearable for any RAM we can get our hands on in this course (even if we decrease block size). We have three obvious options:

1. Sample (randomly or not) fewer sites
2. Annotate sites along their genes and combine them gene-wise
3. Delve into the PyWGCNA code and rewrite the `pickSoftThreshold()` function so that it would consume less memory --> git commit?

Option 3 seems like a nice opportunity for a final project, don't you think? That's why we're skipping it for now.

Let's choose option 2 over option 1, because it will instantly return us to the realm of genes rather than methylation sites, which is exactly the type of data PyWGCNA is used to work with.

How exactly can we do that? If you were on your own, you would have to peruse through the Illumina website to find microarray probe annotations and process them. Luckily for you, we've already done it and provided you with simplified annotations.

In [ ]:
illumina_anno = pd.read_csv(f'{data_dir}/Illumina_850k_v2_annotation_cleaned.csv.gz', index_col=0)
illumina_anno.head(5)

In [ ]:
reprog_sites_anno = illumina_anno.loc[illumina_anno.index.intersection(pyWGCNA_reprog.geneExpr.var.index)]
print(f'Out of {pyWGCNA_reprog.geneExpr.var.shape[0]} sites from the reprogramming dataset, {reprog_sites_anno.shape[0]} were annotated')

reprog_sites_anno.to_csv(f'{data_dir}/reprog_sites_annotation.csv')

Since methylation sites in promoters and gene bodies tend to show different methylation patterns, let's focus our today's analysis on the promoter sites only (i.e., sites located in the vicinity of transcription start sites).

In [ ]:
reprog_sites_anno_promoters = reprog_sites_anno.copy()

reprog_sites_anno_promoters.loc[
    ~reprog_sites_anno_promoters['Group'].astype(str).str.contains('TSS') & \
    ~reprog_sites_anno_promoters['Group_Gencode'].astype(str).str.contains('TSS'), 
    ['Group', 'Group_Gencode']] = np.nan

reprog_sites_anno_promoters.dropna(subset=['Group', 'Group_Gencode'], how='all', inplace=True)

reprog_sites_anno_promoters.to_csv(f'{data_dir}/reprog_sites_promoters_annotation.csv')

reprog_sites_anno_promoters.head(5)

PyWGCNA has a function to retrieve annotation data from Ensembl Biomart, `PyWGCNA.getGeneList()`. But it works so unbearably slow that we chose to do it our own way, using the Python `biomart` module

In [ ]:
# Set up connection to server
server = biomart.BiomartServer('http://ensembl.org/biomart')
mart = server.datasets['hsapiens_gene_ensembl']

# List the types of data we want
attributes = ['ensembl_transcript_id', 'hgnc_symbol', 
              'ensembl_gene_id',  'gene_biotype', 'transcript_biotype']

# Get the mapping between the attributes
response = mart.search({'attributes': attributes})
data = response.raw.data.decode('ascii')

genes_trascripts_df = pd.DataFrame(data.splitlines())
genes_trascripts_df[attributes] = genes_trascripts_df[0].str.split('\t', expand=True)
genes_trascripts_df.drop(0, axis=1, inplace=True)

genes_trascripts_df.head(5)

In [ ]:
# First let's check if there are matching transcript IDs
genes_trascripts_matched1 = genes_trascripts_df.merge(
    reprog_sites_anno_promoters.reset_index(), 
    how='inner', left_on='ensembl_transcript_id', right_on='Transcript_Gencode', sort=False).set_index('index')

# Now let's check if there are matching gene names for what's left out
genes_trascripts_matched2 = genes_trascripts_df.merge(
    reprog_sites_anno_promoters.drop(index=genes_trascripts_matched1.index).reset_index(), 
    how='inner', left_on='hgnc_symbol', right_on='Gene', sort=False).set_index('index')

# And combine them
genes_trascripts_matched = pd.concat([genes_trascripts_matched1, genes_trascripts_matched2], copy=False)

# Drop some probes that got duplicated in the process
genes_trascripts_matched = genes_trascripts_matched.reset_index().drop_duplicates('index', keep='first').set_index('index')

# For some reason, some HGNC symbols retrieved from biomart are empty strings. Let's convert what we can using our Illumina annotation
genes_trascripts_matched['hgnc_symbol'] = genes_trascripts_matched.apply(
    lambda x: x['Gene'] if x['hgnc_symbol']=='' else x['hgnc_symbol'], axis=1)

# Select and save only the gene-related information
cgGeneList = genes_trascripts_matched[['hgnc_symbol', 'ensembl_gene_id', 'gene_biotype']]
cgGeneList.to_csv('./data/cgGeneList.csv')

cgGeneList

In [ ]:
pyWGCNA_reprog.geneExpr

In [ ]:
reprog_metlevs_grouped = pyWGCNA_reprog.geneExpr[:, cgGeneList.index].to_df().T.join(
    cgGeneList['ensembl_gene_id']).groupby('ensembl_gene_id').mean().T

reprog_metlevs_grouped.to_csv(f'{data_dir}/reprog_metlevs_grouped.csv.gz', compression='gzip')
reprog_metlevs_grouped

## Constructing gene network modules (using combined methylation sites)

Here we go again...

In [ ]:
pyWGCNA_reprog_grouped = PyWGCNA.WGCNA(name='metlevs_grouped', 
                                       species='homo sapiens', 
                                       geneExp=reprog_metlevs_grouped, 
                                       outputPath='wgcna_reprog_',
                                       TPMcutoff=0,
                                       save=True)

In [ ]:
pyWGCNA_reprog_grouped.preprocess()

```{admonition} Question 2
:class: dropdown
Compare this dendrogram with what we've got earlier for the ungrouped matrix. Do you notice any changes? Should we remove any samples in this case? **(0.25 pts)**
```

Finally, **now** we get to construct the connectivity network.

Prepare to wait for a while, we're basically performing a huge computational analysis workflow via a single function

In [ ]:
pyWGCNA_reprog_grouped.findModules()

```{admonition} Question 3
:class: dropdown
Did you notice anything odd about how this function ran? *hint*: if not, look carefully through the log. How do you think, what should we do about this behaviour? **(0.25 pts)**
```

```{admonition} Question 4
:class: dropdown
How many modules were identified? What is the main take-away from the last dendrogram? How can we modify WGCNA construction behaviour to change the number of modules retained for analysis? **(0.25 pts)**
```

### Updating sample information and assiging color to them for dowstream analysis

In [ ]:
sampleInfo = pd.read_csv(f'{data_dir}/meta.csv', index_col=0, dtype=str)
pyWGCNA_reprog_grouped.updateSampleInfo(sampleInfo=sampleInfo)
pyWGCNA_reprog_grouped.geneExpr.obs

Add colors for metadata

In [ ]:
sample_types = pyWGCNA_reprog_grouped.geneExpr.obs['SampleType'].unique()
pyWGCNA_reprog_grouped.setMetadataColor('SampleType', {sample_types[i]: sns.color_palette('tab10').as_hex()[i] for i in range(len(sample_types))})

reprog_days = pyWGCNA_reprog_grouped.geneExpr.obs['Day'].astype(str).unique()
pyWGCNA_reprog_grouped.setMetadataColor('Day', {reprog_days[i]: sns.color_palette('viridis', len(reprog_days)).as_hex()[i] for i in range(len(reprog_days))})

### Updating gene information

In [ ]:
# Load genes metadata that we've retrieved from Biomart earlier
cgGeneList = pd.read_csv(f'{data_dir}/cgGeneList.csv', index_col=0)
geneList = cgGeneList.drop_duplicates('ensembl_gene_id').set_index('ensembl_gene_id').rename(columns={'hgnc_symbol': 'gene_name'})

# Add genes metadata to our object
pyWGCNA_reprog_grouped.updateGeneInfo(geneList)

In [ ]:
pyWGCNA_reprog_grouped.geneExpr

**note**: For downstream analysis, we want to keep aside the Grey module (if there is one), which is the collection of genes that could not be assigned to any other module.

### Saving and loading your PyWGCNA

You can save or load your PyWGCNA object with the `saveWGCNA()` or `readWGCNA()` functions respectively.

In [ ]:
pyWGCNA_reprog_grouped.saveWGCNA()

## Relating modules to external information and identifying important genes

PyWGCNA performs some important analysis after identifying modules in `analyseWGCNA()` function including:

1. Quantifying module–trait relationship
2. Gene relationship to trait and modules
3. Gene-ontology analysis

That is why we've added sample and gene information beforehand.

For showing module relationship heatmap, PyWGCNA needs user to choose and set colors from Matplotlib colors for metadata by using `setMetadataColor()` function.

You also can select which data trait in which order you wish to show in module eigengene heatmap

Expect to wait for a while, again...

In [ ]:
pyWGCNA_reprog_grouped = PyWGCNA.readWGCNA("wgcna_reprog_metlevs_grouped.p")
pyWGCNA_reprog_grouped.geneExpr

In [ ]:
pyWGCNA_reprog_grouped.analyseWGCNA()

In [ ]:
pyWGCNA_reprog_grouped.saveWGCNA()

```{admonition} Question 5
:class: dropdown
That's a lot of heatmaps and barplots. What can you assume about the dynamics of promoter DNA methylation modules across cellular reprogramming based on these plots (and on what you know about reprogramming and embryonic stem cells)? (0.5 pts)

*hint*: Check the `./wgcna_reprog_figures/GO folder`, you might find something useful in there
```

## Finding hub genes for each modules

You can also find hub genes in each module based on their connectivity by using the `top_n_hub_genes()` function.

It will give you dataframe sorted by connectivity along with the additional gene information you have in your metadata.

In [ ]:
modules = pyWGCNA_reprog_grouped.datExpr.var.moduleColors.unique().tolist()

In [ ]:
pyWGCNA_reprog_grouped.top_n_hub_genes(moduleName=modules[1], n=10)

# Part II. Functional enrichment analysis using PyWGCNA

After finding coexpression modules, you can investigate each module separately using functional enrichment analysis. PyWGCNA supports this directly from the GO, KEGG, and REACTOME databases using the `functional_enrichment_analysis()` function.

Let's get straight into it!

In [ ]:
# Reload PyWGCNA object you've made if you've lost connection or something else...
pyWGCNA_reprog_grouped = PyWGCNA.readWGCNA("wgcna_reprog_metlevs_grouped.p")
pyWGCNA_reprog_grouped.datExpr.var.moduleColors.value_counts()

## Gene ontology (GO) analysis
Say we want to investigate the black module. After defining which gene set libraries we want to use, (pick from [here](https://maayanlab.cloud/Enrichr/#libraries)), we make the following call:

In [ ]:
moduleName = 'black'

In [ ]:
pyWGCNA_reprog_grouped.figureType = "png"
gene_set_library = ["GO_Biological_Process_2023", "GO_Cellular_Component_2023", "GO_Molecular_Function_2023"]
pyWGCNA_reprog_grouped.functional_enrichment_analysis(type="GO",
                                                      moduleName=moduleName,
                                                      sets=gene_set_library,
                                                      p_value=1,
                                                      file_name=f"GO_{moduleName}_2023")

In [ ]:
pyWGCNA_reprog_grouped.figureType = "png"
gene_set_library = ["KEGG_2021_Human"]
pyWGCNA_reprog_grouped.functional_enrichment_analysis(type="KEGG",
                                                      moduleName=moduleName,
                                                      sets=gene_set_library,
                                                      p_value=0.05)

In [ ]:
pyWGCNA_reprog_grouped.figureType = "pdf"
pyWGCNA_reprog_grouped.functional_enrichment_analysis(type="REACTOME",
                                                      moduleName=moduleName,
                                                      p_value=0.05)

```{admonition} Exercise 1
:class: dropdown
Repeat GO analysis for the other modules and describe the outcomes. What else could we do to perform functional enrichment? (0.5 pts)
```

```{admonition} Exercise 2
:class: dropdown
Using top-10 hub genes, try to make an informed guess about the functional enrichments of each module (0.5 pts)
```

# Part III. Visualizing networks with PyWGCNA

After you finding your modules, you can plot each module or all modules together as a network.

You can plot each module as a network using the `CoexpressionModulePlot()` function. This will save the plot as an html file in the output directory figures/network with the module name. For this example, it will save at figures/network/black.html.

You can define the number of genes and connections you want to see and the minimum TOM value to be considered a connection or not.

The HTML file is an interactive network so if you click on any nodes you can see additional information about each node (gene).


In [ ]:
pyWGCNA_reprog_grouped.CoexpressionModulePlot(modules=[moduleName], numGenes=10, numConnections=100, minTOM=0)


You can also filter genes based on the information you have in `datExpr.var`. Imagine we only want to see protein coding genes in the black module.

In [ ]:
filters = {"gene_biotype": ["protein_coding"]}
pyWGCNA_reprog_grouped.CoexpressionModulePlot(modules=[moduleName], filters=filters, file_name=f"{moduleName}_protein_coding")

```{admonition} Exercise 3
:class: dropdown
Draw coexpression plots for every module using protein coding genes only and try to assume biological meaning behind each module's networks (1 pt)
```

If you want to display a network for more than one module and to show the connections between each module, you can simply provide a list of modules to the `CoexpressionModulePlot()` function.

```{admonition} Exercise 4
:class: dropdown
Draw coexpression plots for all modules combined. You might find it useful to play around with filters and with the `numGenes` and `numConnections` variables. Don't forget to set `file_name="all"` to save it as a separate file (1.5 pts)
```

# Part IV. The real (home)work

```{admonition} Exercise 5
:class: dropdown
Perform weighted gene-coexpression network analysis for the CpG sites (in our reprogramming dataset) located outside of promoters and try to make some conclusions about the dynamics of DNA methylation in all modules that you've managed to find, both in promoters and non-promoters. You are expected to provide these conclusions as a list of no less than three concise bullet points (5 pts)
```